# HKU QIDS 2023 Quantitative Investment Competition: Model

## Init Config

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from qids_package.qids import *
import warnings
from submit import submit

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

warnings.filterwarnings("ignore")

In [69]:
seed = 257248
stock_num = 54
day_num_total = 1000
day_num = 1000 - 2
test_day_num = 700
timeslot_num = 50

In [70]:
def std(train, valid, test=None):
    scaler = StandardScaler()
    scaler.fit(train)
    train = scaler.transform(train)
    valid = scaler.transform(valid)
    if test is not None:
        test = scaler.transform(test)
    return train, valid, test

In [71]:
def calc_corr(df1, df2):
    return np.corrcoef(df1, df2)[0][1]

In [72]:
# def evaluate(model, train, valid, test, train_y, valid_y, return_pred=True, version=2, return_auc=False, plot_auc=False):
#     model.fit(train, train_y)
#     if version == 2:
#         model_train_y = model.predict(train)
#         model_valid_y = model.predict(valid)
#         pred = model.predict(test)
#         acr_train = model.score(train, train_y)
#         acr_valid = model.score(valid, valid_y)
#         print(calc_corr(train_y, model_train_y))
#         print(calc_corr(valid_y, model_valid_y))
#         if return_pred:
#             return pred
    #     cm = confusion_matrix(train_y, model_train_y)
    #     tn, fp, fn, tp = cm.ravel()
    #     print(f"Train AUROC: {roc_auc_score(train_y, model_train_y):.4f}; FP: {fp:>4d}; FN: {fn:>4d}")
    #     cm = confusion_matrix(valid_y, model_valid_y)
    #     tn, fp, fn, tp = cm.ravel()
    #     print(f"Valid AUROC: {roc_auc_score(valid_y, model_valid_y):.4f}; FP: {fp:>4d}; FN: {fn:>4d}")
    #     print()
    # elif version == 3:
    #     pred_train = model.predict(train)
    #     pred_valid = model.predict(valid)
    #     pred = model.predict(test)
    #     print(f"Train RMSE: {mse(train_y, pred_train, squared=False):.4f}")
    #     print(f"Valid RMSE: {mse(valid_y, pred_valid, squared=False):.4f}")
    # else:
    #     model_train_y = model.predict_proba(train)[:,1]
    #     model_valid_y = model.predict_proba(valid)[:,1]
    #     pred = model.predict_proba(test)[:,1]
    #     acr_train = model.score(train, train_y)
    #     acr_valid = model.score(valid, valid_y)
    #     print(f"Train Accuracy: {acr_train:.4f}; Validation Accuracy: {acr_valid:.4f}")
    #     auc_train = roc_auc_score(train_y, model_train_y)
    #     auc_valid = roc_auc_score(valid_y, model_valid_y)
    #     print(f"Train AUROC: {auc_train:.4f}")
    #     print(f"Valid AUROC: {auc_valid:.4f}")
    #     print()
    # if plot_auc:
    #     try:
    #         fpr1, tpr1, thresholds1 = roc_curve(train_y, model_train_y)
    #         fpr2, tpr2, thresholds2 = roc_curve(valid_y, model_valid_y)
    #         plt.plot([0,1], 'k--')
    #         plt.plot(fpr1, tpr1, label= "Train")
    #         plt.plot(fpr2, tpr2, label= "Valid")
    #         plt.legend()
    #         plt.xlabel("FPR")
    #         plt.ylabel("TPR")
    #         plt.title('AUROC Curve')
    #         plt.show()
    #     except:
    #         pass
    # if return_pred and return_auc and version == 1:
    #     return pred, auc_train, auc_valid
    # elif return_pred:
    #     return pred
    # else:
    #     return

In [135]:
def evaluate(model, train, test, train_y, real_y):
    model.fit(train, train_y)
    model_train_y = model.predict(train)
    pred = model.predict(test)
    # return pred
    # pred = np.array(model.predict(test)).reshape(-1,1)
    # pred = pred[:37692]
    # real_y = np.array(real_y).reshape(-1,1)
    # print(calc_corr(train_y, model_train_y))
    # print(calc_corr(real_y, pred))
    # print(real_y.shape)
    # print(pred.shape)
    # print("train_score",calc_corr(train_y, model_train_y))
    # print("prediction_score",calc_corr(pred, real_y))
    # train_y = pd.DataFrame(train_y)
    # model_train_y = pd.DataFrame(model_train_y)
    # combined0 = pd.concat([train_y,model_train_y],axis = 1)
    # print(combined0.corr())
    # pred_temp = pd.DataFrame(pred_temp)
    # real_y = pd.DataFrame(real_y)
    # combined = pd.concat([pred_temp,real_y],axis = 1)
    # print(combined.corr())
    # dt = pd.read_csv("../sub/sample_submission.csv")
    # dt["stock_id"] = dt["date_time"].apply(lambda x: x.split("d")[0][1:]).astype("int")
    # dt["day"] = dt["date_time"].apply(lambda x: x.split("d")[1][:]).astype("int")
    # dt = dt.drop(columns=["return"]).sort_values(by=["stock_id", "day"])
    # dt = dt.reset_index(drop=True)
    # pred = pd.concat([dt, pd.DataFrame(pred, columns=["return"])], axis=1)
    # pred = pred.sort_values(by=["day", "stock_id"]).reset_index(drop=True)["return"]
    print(calc_corr(train_y, model_train_y))
    print(calc_corr(real_y, pred[:37692]))
    # pred_subset = []
    # for stock in range(stock_num):
    #     stock_info = pred[stock*test_day_num:(stock+1)*test_day_num-2]
    #     pred_subset.append(stock_info)
    # pred_subset = np.array(pred_subset).reshape(-1,1).squeeze(1)
    # print(calc_corr(np.array(real_y), pred_subset))
    return pred

In [177]:
def evaluate2(model, train, test, train_y, real_y):
    model.fit(train, train_y)
    model_train_y = model.predict(train)
    pred = model.predict(test)
    # print(calc_corr(train_y, model_train_y))
    # print(calc_corr(real_y, pred[:698]))
    return pred

## Load Data

In [137]:
write_path = "../data/"

train_path = write_path + "train.csv"
test_path = write_path + "test.csv"
real_return_path = write_path + "real_return.csv"
# real_return_path = write_path + "real_return_reorder.csv"

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
real_return = pd.read_csv(real_return_path)

In [138]:
train_y = train["return"]
train = train.drop(columns=["return", "date_time", "stock_id", "day"])

test = test.drop(columns=["date_time", "stock_id", "day"])

real_y = real_return["return"]

## Normalization

In [21]:
# train, valid, test = std(train.iloc[:, 3:-1], valid.iloc[:, 3:-1], test.iloc[:, 3:])

## Model

### Linear Regression

In [149]:
model = LinearRegression()
pred = evaluate(model, train, test, train_y, real_y)

0.08185975427789273
0.040371253903712594


### Ridge Regression

In [140]:
model = Ridge(alpha=1e8)
pred = evaluate(model, train, test, train_y, real_y)

0.07108389523027638
0.05978931377050287


### Lasso Regression

In [146]:
model = Lasso(alpha=1)
pred = evaluate(model, train, test, train_y, real_y)

0.07082556523046829
0.05973729199810845


In [216]:
# [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6,1e-7,1e-8,1e-9,1e-10]
# [1e1,1e2,1e3,1e4,1e5,1e6,1e7,1e8,1e9,1e10]
for i in [0.012]:
    result = None
    for stock in range(stock_num):
        start = stock * day_num
        end = start + day_num
        model = Ridge(alpha=i, normalize=True)
        pred = evaluate2(
            model, 
            train.iloc[stock*day_num:(stock+1)*day_num, :], 
            test.iloc[stock*test_day_num:(stock+1)*test_day_num, :], 
            train_y.iloc[stock*day_num:(stock+1)*day_num], 
            real_y.iloc[stock*test_day_num:(stock+1)*test_day_num-2]
            # real_y.iloc[[i for i in range(stock, len(real_y), stock_num)]]
        )
        if result is None:
            result = pred
        else:
            result = np.concatenate([result, pred], axis=0)
    print(i, calc_corr(result[:37692], real_y))

0.012 0.07225325752784392


## Submission

In [88]:
submit(pred)